<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-Decorators" data-toc-modified-id="Python-Decorators-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python Decorators</a></span><ul class="toc-item"><li><span><a href="#Timer-decorator" data-toc-modified-id="Timer-decorator-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Timer decorator</a></span></li><li><span><a href="#Python-matrix-iterator-decorator" data-toc-modified-id="Python-matrix-iterator-decorator-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Python matrix iterator decorator</a></span></li></ul></li></ul></div>

# Python Decorators 

A decorator is a design pattern in Python that allows a user to add new code to an existing function, method, or class without directly impacting subclasses or impacting the source code of a function beeing decorated.

In [1]:
def my_decorator(func):
    def wrapper():
        print("Logic before func is called.")
        func()
        print("Logic after func is called.")
    return wrapper

def my_func():
    print("my_func is executed")

deco_my_func = my_decorator(my_func)

In [2]:
deco_my_func()

Logic before func is called.
my_func is executed
Logic after func is called.


The function decoration happens when calling `deco_my_func = my_decorator(my_func)`.

Note that `my_decorator` returns a function. The function that returns is a wrapper to the input function `func`  that `my_decorator` recieves.

Instead of writting

```
deco_my_func = my_decorator(my_func)
```

if a function is not meant to be used without the modification done by a decorator  there is the option of define a function with the decorator already in place using the notation `@my_decorator`.

In [3]:
@my_decorator
def my_func():
    print("my_func is executed")

In [4]:
my_func()

Logic before func is called.
my_func is executed
Logic after func is called.


## Timer decorator

In [16]:
import time

def timefunc(func):
    """Print the runtime of the decorated function"""
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()   
        value = func(*args, **kwargs)
        end_time = time.perf_counter()     
        run_time = end_time - start_time   
        print(f"{func.__name__!r} takes {run_time:.4f} secs")
        return value
    return wrapper

def my_func(n):
    aux = []
    for _ in range(n):
        r = sum([i**2 for i in range(n)])
        aux.append(r)
    return sum(aux)

In [17]:
my_func(100)

32835000

We can time a function using

```
timefunc(some_function)(some_function_arguments)
```


In [22]:
result = timefunc(my_func)(500)
result

'my_func' takes 0.0670 secs


20770875000

or we can simply do 

In [23]:
@timefunc
def my_func(n):
    aux = []
    for _ in range(n):
        r = sum([i**2 for i in range(n)])
        aux.append(r)
    return sum(aux)

In [24]:
my_func(100)

'my_func' takes 0.0023 secs


32835000

## Python matrix iterator decorator

In [122]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
import torch

Different matrices allow different `slicing mechanisms` or iterator mechanisms.

Imagine you want to iterate over rows of an `array` and print each row.

Different types of arrays might access differently the rows. Let us see an example

In [125]:
def print_rows(X):
    n_rows = X.shape[0]
    for i in range(n_rows):
        print(f'row {i} is {X[i]}')

In [126]:
print_rows(X_np)

row 0 is [0.69646919 0.28613933]
row 1 is [0.22685145 0.55131477]
row 2 is [0.71946897 0.42310646]


In [135]:
print_rows(X_csr)

row 0 is   (0, 0)	0.6964691855978616
  (0, 1)	0.28613933495037946
row 1 is   (0, 0)	0.2268514535642031
  (0, 1)	0.5513147690828912
row 2 is   (0, 0)	0.7194689697855631
  (0, 1)	0.42310646012446096


In [144]:
#torch.sparse.FloatTensor(X_csr.indices,X_csr.data)

In [124]:
np.random.seed(123)
X_np = np.random.rand(3,2)
X_coo = sp.coo_matrix(X_np)
X_csr = sp.csr_matrix(X_np)
X_tf = tf.constant(X_np)
X_pytorch = torch.tensor(X_np)

In [128]:
import scipy

def matrix_iterator(X):
    n_rows = X.shape[0]
    for i in range(n_rows):
        if scipy.sparse.issparse(X):
            yield X.getrow(i)
        else:
            yield X[i]

In [129]:
for i,x in enumerate(matrix_iterator(X_np)):
    print(f'row {i} from X = {x}')

row 0 from X = [0.69646919 0.28613933]
row 1 from X = [0.22685145 0.55131477]
row 2 from X = [0.71946897 0.42310646]


In [130]:
for i,x in enumerate(matrix_iterator(X_csr)):
    print(f'row {i} from X = {x}')

row 0 from X =   (0, 0)	0.6964691855978616
  (0, 1)	0.28613933495037946
row 1 from X =   (0, 0)	0.2268514535642031
  (0, 1)	0.5513147690828912
row 2 from X =   (0, 0)	0.7194689697855631
  (0, 1)	0.42310646012446096


In [131]:
for i,x in enumerate(matrix_iterator(X_coo)):
    print(f'row {i} from X = {x}')

row 0 from X =   (0, 1)	0.28613933495037946
  (0, 0)	0.6964691855978616
row 1 from X =   (0, 1)	0.5513147690828912
  (0, 0)	0.2268514535642031
row 2 from X =   (0, 1)	0.42310646012446096
  (0, 0)	0.7194689697855631


In [133]:
for i,x in enumerate(matrix_iterator(X_tf)):
    print(f'row {i} from X = {x}')

row 0 from X = [0.69646919 0.28613933]
row 1 from X = [0.22685145 0.55131477]
row 2 from X = [0.71946897 0.42310646]


In [157]:
for i,x in enumerate(matrix_iterator(X_pytorch)):
    print(f'row {i} from X = {x}')

row 0 from X = tensor([0.6965, 0.2861], dtype=torch.float64)
row 1 from X = tensor([0.2269, 0.5513], dtype=torch.float64)
row 2 from X = tensor([0.7195, 0.4231], dtype=torch.float64)


In [159]:
np.random.seed(123)
X_np = np.random.rand(3,2)
X_coo = sp.coo_matrix(X_np)
X_csr = sp.csr_matrix(X_np)

In [160]:
X_csr.shape

(3, 2)

In [154]:
scipy.sparse.vstack((X_csr,X_csr[1,:]))

<4x2 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>